In [1]:
# import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import glob
import csv
import os
import random
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
# Access data

def get_data(data_dir, labels_path):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    num_examples = len(data_filenames)

    max_length = 0
    for i, df in enumerate(data_filenames):
        max_length = max(max_length, np.load(df).shape[0])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length])
    Y = np.zeros(num_examples)
    for i, df in enumerate(data_filenames):
        data = np.load(df)
        X[i, :data.shape[0]] = data
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_random_data(data_dir, labels_path, num_examples):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    max_length = 0
    for i in range(num_examples):
        df = data_filenames[i]
        max_length = max(max_length, np.load(df).shape[0])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length])
    Y = np.zeros(num_examples)
    for i in range(num_examples):
        df = data_filenames[i]
        
        data = np.load(df)
        X[i, :data.shape[0]] = data
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_random_spect_data(data_dir, labels_path, num_examples=None):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    max_width = 0
    max_length = 0
    if num_examples is None:
        for i, df in enumerate(data_filenames):
            max_width = max(max_width, np.load(df).shape[0])
            max_length = max(max_length, np.load(df).shape[1])
        num_examples = i + 1
    else:
        for i in range(num_examples):
            df = data_filenames[i]
            max_width = max(max_width, np.load(df).shape[0])
            max_length = max(max_length, np.load(df).shape[1])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length, max_width])
    Y = np.zeros(num_examples)
    for i in range(num_examples):
        df = data_filenames[i]
        
        data = np.load(df)
        X[i, :data.shape[1], :data.shape[0]] = data.T
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_labels_dict(reference_path, reference):
    with open(reference_path) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            reference[row[0]] = row[1]


In [3]:
# Methods to account for variable sequence lengths (call for each batch)
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    length = tf.reduce_sum(used, 1)
    length = tf.cast(length, tf.int32)
    return length

def last_relevant(output, length):
    batch_size = tf.shape(output)[0]
    max_length = tf.shape(output)[1]
    out_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, out_size])
    relevant = tf.gather(flat, index)
    
    return relevant

In [4]:

def _variable_on_cpu(name, shape, initializer):
    """
    Helper to create a Variable stored on CPU memory.
    """
    
    with tf.device('/cpu:0'):
        var = tf.get_variable(
            name=name, 
            shape=shape, 
            dtype=tf.float32, 
            initializer=initializer)

    return var

def _variable_with_weight_decay(name, shape, stddev, wd):
    """
    Helper to create an initialized Variable with weight decay.
    """

    var = _variable_on_cpu(
        name=name,
        shape=shape,
        initializer=tf.truncated_normal_initializer(
            stddev=stddev,
            dtype=tf.float32))

    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, 
                                   name='weight_loss')
        tf.add_to_collection('losses', weight_decay)

    return var


def loss(unscale_logits, labels):
    """
    Add L2Loss to all the trainable variables.
    """

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels,
        logits=unscale_logits,
        name='cross_entropy')

    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

    tf.add_to_collection('losses', cross_entropy_mean)

    total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

    return total_loss

In [5]:
def rnn(features):
    
    features = features['x']
    
    # Input Layer
    input_layer = w

    # RNN layer #1
    with tf.variable_scope(name_or_scope='rnn') as scope:
        n_units = 500
#         base_cell = tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) tf.nn.rnn_cell.BasicLSTMCell
#         rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) for _ in range(2)])
#         rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(n_units) for _ in range(2)])

#         outputs, rnn_state =  tf.nn.dynamic_rnn(rnn_cell, 
#                                            inputs=input_layer,
#                                            # initial_state=initial_state,
#                                            dtype=tf.float32,
#                                            # sequence_length=length(input_layer),
#                                            time_major=False,
#                                            scope=scope.name)
        
        rnn_cell_fw = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(n_units) for _ in range(2)])
        rnn_cell_bw = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(n_units) for _ in range(2)])

        outputs, rnn_state =  tf.nn.bidirectional_dynamic_rnn(cell_fw=rnn_cell_fw,
                                                              cell_bw=rnn_cell_bw,
                                                              inputs=input_layer,
                                                              dtype=tf.float32,
                                                              time_major=False,
                                                              scope=scope.name)
        
        outputs = tf.concat(outputs, 2)
        
        
    # Fully connected layer #1
    with tf.variable_scope(name_or_scope='fc1') as scope:

        weights = _variable_with_weight_decay(
            name='weights',
            shape=[2 * n_units, 250], # needs to match output dimension of RNN
            stddev=0.04,
            wd=0.004)

        biases = _variable_on_cpu(
            name='biases',
            shape=[250], # needs to match output dimension of RNN
            initializer=tf.constant_initializer(0.1))

        pre_activation = tf.add(tf.matmul(outputs[:, -1, :], weights), biases)

        fc1 = tf.nn.relu(
            features=pre_activation,
            name=scope.name)
        
    # Fully connected layer #2
    with tf.variable_scope(name_or_scope='fc2') as scope:

        weights = _variable_with_weight_decay(
            name='weights',
            shape=[250, 2], # needs to match output dimension of RNN
            stddev=0.04,
            wd=0.004)

        biases = _variable_on_cpu(
            name='biases',
            shape=[2], # needs to match output dimension of RNN
            initializer=tf.constant_initializer(0.1))

        pre_activation = tf.add(tf.matmul(fc1, weights), biases)

        fc2 = tf.nn.relu(
            features=pre_activation,
            name=scope.name)

    return fc2

# def train_neural_network(): 
#     prediction = recurrent_neural_network(x_placeholder)
#     cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=tf.reshape(y_placeholder, [batch_size, n_classes])))
#     optimizer = tf.train.AdamOptimizer().minimize(cost)
#     epoch_batch_itr = 10
    
#     correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_placeholder, 1))
#     accuracy = tf.reduce_mean(tf.cast(correct, 'float32'))
        
#     with tf.Session() as sess:
#         tf.global_variables_initializer().run()

#         for epoch in range(hm_epochs):
#             epoch_loss = 0
#             for b in range(epoch_batch_itr):
#                 batchX, batchY = generate_batch(train_path)
#                 _, c = sess.run([optimizer, cost], feed_dict={x_placeholder: batchX, y_placeholder: batchY})
#                 epoch_loss += c
#             validation_batchX, validation_batchY = generate_batch(validation_path) 
#             print('Epoch', epoch, 'loss:', epoch_loss, 'Validation Accuracy:', accuracy.eval({x_placeholder: validation_batchX, y_placeholder: validation_batchY}))        

In [6]:

def rnn_model_fn(features, labels, mode, params):

    """
    Build model.
    """
    model = params['model']
#     unscale_logits = rnn(features)
    unscale_logits = model(features)
    
    # Generate predictions for PREDICT and EVAL modes.
    predictions = {
        'classes': tf.argmax(input=unscale_logits, axis=1),
        'probabilities': tf.nn.softmax(unscale_logits, name='softmax_tensor')
    }
    
    ####################
    # PREDICT
    ####################
    if mode == tf.estimator.ModeKeys.PREDICT: 
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions['probabilities'])
    
    else:
        # Calculate loss for both TRAIN and EVAL modes.
        labels = tf.cast(labels, tf.int64)
        total_loss = loss(unscale_logits, labels) #REFORM THIS LOSS FUNCTION

        # Add summary operation for total loss visualizaiton.
        tf.summary.scalar(
            name='total_loss',
            tensor=total_loss)
        
        
        ####################
        # TRAIN
        ####################
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            # Compute gradients using Gradient Descent Optimizer.
            optimizer = tf.train.AdamOptimizer()

            grads_vars = optimizer.compute_gradients(loss=total_loss)

            # Add summary operations for gradient visualizations.
            for grad, var in grads_vars:
                if grad is not None:
                    tf.summary.histogram(
                        name=var.op.name + '/gradients', 
                        values=grad)

            train_op = optimizer.minimize(
                loss=total_loss,
                global_step=tf.train.get_global_step())

            # Add evaluation metrics for TRAIN mode.
            accuracy_train = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for training accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_train',
                tensor=accuracy_train[0])

            train_summary_hook = tf.train.SummarySaverHook(
                save_steps=10,
                output_dir='models/rnn',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss, 
                train_op=train_op,
                training_hooks=[train_summary_hook])
        
        
        ####################
        # EVALUATE
        ####################
        else:
            accuracy_valid = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for validation accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_validation',
                tensor=accuracy_valid[0])

            eval_metric_ops = {"accuracy": accuracy_valid}

            eval_summary_hook = tf.train.SummarySaverHook(
                save_steps=1,
                output_dir='models/rnn',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode, 
                loss=total_loss, 
                eval_metric_ops=eval_metric_ops,
                training_hooks=[eval_summary_hook])
        

In [7]:
# Main function for building, training and evaluating model.
def main(train_data, train_labels, eval_data, eval_labels, test_data):
    
    estimator_dir = 'models/rnn'
    
    # Delete directory containing events logs and checkpoints if it exists.
    if tf.gfile.Exists(estimator_dir):
        tf.gfile.DeleteRecursively(estimator_dir)

    # Create directory containing events logs and checkpoints.
    tf.gfile.MakeDirs(estimator_dir)
    
    # Create the Estimator.
    classifier = tf.estimator.Estimator(
        model_fn=rnn_model_fn, 
        model_dir=estimator_dir,
        params={'model': rnn})

#     for _ in range(100):
    for _ in range(2):
        
        # Train the model.
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={'x': train_data},
            y=train_labels,
#             batch_size=128,
            batch_size=1,
            num_epochs=None,
            shuffle=True)

        classifier.train(
            input_fn=train_input_fn,
#             steps=50)
            steps=2)

        # Evaluate the model and print results.
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            y=eval_labels,
#             batch_size=,
            num_epochs=1,
            shuffle=False)
    
        eval_results = classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)
        
    # Generate predictions on test set.
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      num_epochs=1,
      shuffle=False)

    predictions = np.array(list(classifier.predict(input_fn=predict_input_fn))).T
    
    return predictions
        

In [8]:
# data_dir = 'data/sequence'

# X_train, Y_train = get_data(os.path.join(data_dir, 'training/sub'), 
#                             os.path.join(data_dir, 'REFERENCE.csv'))
                            
# X_valid, Y_valid = get_data(os.path.join(data_dir, 'validation/sub'), 
#                             os.path.join(data_dir, 'REFERENCE.csv'))

In [9]:
# Load training data

data_dir = '/Volumes/light/deeplearning_proj/data/spectrogram/training/sub_ws300'
labels_path = '/Volumes/light/deeplearning_proj/data/spectrogram/training/REFERENCE.csv'
num_examples = 1000

X_train, Y_train = get_random_spect_data(data_dir, labels_path, num_examples)

# Load validation data
data_dir = '/Volumes/light/deeplearning_proj/data/spectrogram/validation/sub_ws300'
labels_path = '/Volumes/light/deeplearning_proj/data/spectrogram/validation/REFERENCE.csv'

X_valid, Y_valid = get_random_spect_data(data_dir, labels_path)

In [10]:
pred = main(X_train, Y_train, X_valid, Y_valid, X_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/rnn', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 1.3314, step = 1
INFO:tensorflow:Saving checkpoints for 2 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 1.19251.
INFO:tensorflow:Starting evaluation at 2017-12-10-00:21:19
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-2
INFO:tensorflow:Finished evaluation at 2017-12-10-00:21:41
INFO:tensorflow:Saving dict for global step 2: accuracy = 0.489399, global_step = 2, loss = 1.37614
{'accuracy': 0.48939928, 'loss': 1.3761444, 'global_step': 2}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:ten

In [ ]:
accu = [0.6042403,0.65459365,0.66431093,0.63427562,0.61749119,0.63074207,0.61042404,0.62720847,0.6448763,0.63515902,0.61925793,0.64045936,0.62102473,0.62544167,0.62632507,0.66342759,0.62014133,0.62544167,0.6033569,0.61484098,0.63515902,0.62014133,0.63604242,0.6492933,0.5998233,0.62809187,0.61484098,0.6448763,0.63692582,0.63692582,0.64045936,0.63692582,0.64045936,0.64134276,0.63869256,0.63869256,0.64222616,0.63957596,0.64222616,0.61307418,0.62455833,0.6051237,0.63162541,0.6042403,0.6475265,0.62897527,0.63427562,0.64134276,0.6475265,0.63604242,0.63427562,0.62897527,0.62897527,0.63250881,0.63074207,0.62720847,0.63957596,0.63515902,0.63780922,0.64310956,0.62367493,0.6024735,0.59363955,0.5998233,0.59363955,0.66342759,0.62632507,0.64045936,0.63339221,0.63515902,0.64134276,0.64134276,0.63957596,0.63692582,0.63780922,0.61307418,0.62014133,0.58745581,0.61484098,0.62190813,0.65194345,0.6448763,0.63957596,0.64222616,0.6484099,0.6492933,0.64310956,0.64134276,0.63957596,0.6466431,0.63604242,0.63427562,0.63957596,0.63604242,0.59363955,0.63692582,0.62632507,0.62897527,0.63339221,0.63515902]
plt.plot(accu)
plt.ylim(0., 1.0)
plt.show()